In [ ]:
import skimage.measure as msr
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
from PIL import Image, ImageCms

import glob
import os
from pathlib import Path
import imageio
import cv2

In [ ]:
# Importing class from wmark.py
from wmark import WaterMark

In [ ]:
# Source Directory
srcFolder = 'TestSet/'
# Source Path
srcPth = Path(srcFolder).resolve()

# Destination Directory
# dstFolder = Path(srcFolder+'/marked/').resolve()
# Creating folder for marked images
# Path(dstFolder).mkdir(exist_ok=True)
# Destination Path
# dstPath = Path(dstFolder).resolve()

In [ ]:
# Creating empty DataFrame that will fill up after the loop
metricDataframe = pd.DataFrame()

for i in range(1, 2):
    
    # Generating random seed as integer up to 10000
    randomNums = np.random.randint(10000, size=(1))
    randomNum = randomNums[0]
    print(randomNum)

    wObject = WaterMark(randomNum)

    # All TIF files in the src_path are now imgs
    imgsSource = srcPth.glob('*.tif')
    
    # Counting the number of images in src_folder
    list = os.listdir(srcFolder)
    # -1 because there os one hidden file in directory
    numImages = len(list) - 1
    
    # Creating zeros Numpy Array and setting counter for results
    results = np.zeros([numImages, 9])
    counter = 0
        

    for img in imgsSource:
    
        # Read original image
        imgOriginal = WaterMark.imread(img)
        
        # Find impact factor within a PSNR range
        ImpactFactor = wObject.findImpactFactor(imgOriginal, rangePSNR = (35,40))
        
        # Mark image
        imgMarked = wObject.embedMark(imgOriginal, factor = ImpactFactor)
        
        
        #CROPPING
        #CROPPING
        #CROPPING
        
        # Width
        lx = imgMarked.shape[0]
        # Height
        ly = imgMarked.shape[1]
        
        # Generating cropped image
        # 1/8 from each side (result is by 1/4 smaller than original)
        imgCropped = imgMarked[lx // 8: - lx // 8, ly // 8: - ly // 8]
        
        
        #SCALING
        #SCALING
        #SCALING
        
        # Wanted value of scaled image (in pixels)
        height = 768
        width = 768
        
        # Generating scaled image
        imgScaled = cv2.resize(imgMarked, (width, height))

        
        #ROTATION
        #ROTATION
        #ROTATION
        
        # Rotation angle
        angle = 90
        
        # Defines center of image
        rows, cols = imgMarked.shape[:2]
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        
        # Generating rotated image
        imgRotated = cv2.warpAffine(imgMarked, M, (cols, rows))    

        
        # AFFINE TRANSFORMATION
        # AFFINE TRANSFORMATION
        # AFFINE TRANSFORMATION
        
        # Defining points in original image
        point1a = [50, 50]
        point1b = [200, 50]
        point1c = [50, 200]
        # Defining points in transformed image
        point2a = [10, 100]
        point2b = [200, 50]
        point2c = [100, 250]
        
        rows, cols, ch = imgMarked.shape
        # Compiling original points into single array
        pts1 = np.float32([point1a, point1b, point1c])
        # Compiling transformed points into single array
        pts2 = np.float32([point2a, point2b, point2c])
        M = cv2.getAffineTransform(pts1, pts2)
        
        #Generating affined image
        imgAffined = cv2.warpAffine(imgMarked, M, (cols, rows))
       
        
        # 2D CONVOLUTION
        # 2D CONVOLUTION
        # 2D CONVOLUTION
        
        # Horizontal deviation (intensity)
        kernelX = 3
        # Vertical deviation (intensity)
        kernelY = 3
        
        kernelMultiplication = kernelX * kernelY
        kernel = np.ones((kernelX, kernelY), np.float32)/kernelMultiplication
        
        #Generating convoluted image
        imgConvoluted = cv2.filter2D(imgMarked, -1, kernel)
       
        
        # GAUSSIAN BLUR
        # GAUSSIAN BLUR
        # GAUSSIAN BLUR
        
        # Horizontal deviation (blur intensity)
        kernelX = 3
        # Vertical deviation (blur intensity)
        kernelY = 3
        
        # Generating blurred image
        imgBlurred = cv2.GaussianBlur(imgMarked, (kernelX, kernelY), cv2.BORDER_DEFAULT)
  
        
        # GAUSSIAN NOISE
        # GAUSSIAN NOISE
        # GAUSSIAN NOISE
        
        # Wanted value of rotation angle (in degrees)
        noiseAmount = 0.5
        
        # Generate Gaussian noise
        gauss = np.random.normal(0, noiseAmount, imgMarked.size)
        gauss = gauss.reshape(
            imgMarked.shape[0], imgMarked.shape[1], imgMarked.shape[2]).astype('uint8')
        
        # Generating Gaussian noised image
        imgNoised = cv2.add(imgMarked, gauss)
      
        
        metricOriginal = wObject.decodeMark(imgOriginal, 'CORR')
        results[counter, 0] = metricOriginal
        
        metricMarked = wObject.decodeMark(imgMarked, 'CORR')
        results[counter, 1] = metricMarked
        
        metricCropped = wObject.decodeMark(imgCropped, 'CORR')
        results[counter, 2] = metricCropped
        
        metricScaled = wObject.decodeMark(imgScaled, 'CORR')
        results[counter, 3] = metricScaled
        
        metricRotated = wObject.decodeMark(imgRotated, 'CORR')
        results[counter, 4] = metricRotated
        
        metricAffined = wObject.decodeMark(imgAffined, 'CORR')
        results[counter, 5] = metricAffined
        
        metricConvoluted = wObject.decodeMark(imgConvoluted, 'CORR')
        results[counter, 6] = metricConvoluted
        
        metricBlurred = wObject.decodeMark(imgBlurred, 'CORR')
        results[counter, 7] = metricBlurred
        
        metricNoised = wObject.decodeMark(imgNoised, 'CORR')
        results[counter, 8] = metricNoised
        
        counter += 1
        
        
        metricValues = ["Original", "Marked", "Cropped", "Scaled", "Rotated", "Affined", "2D Convoluted", "Blurred", "Noised"]
        metricSeeds = [f"{randomNum}", f"{randomNum}", f"{randomNum}", f"{randomNum}", f"{randomNum}", f"{randomNum}", f"{randomNum}", f"{randomNum}", f"{randomNum}"]
        
        finalMetric = pd.DataFrame(results, columns = [metricSeeds, metricValues])
    

    metricDataframe = pd.concat([metricDataframe, finalMetric], axis = 1)

In [ ]:
# Creating empty DataFrame that will fill up after the loop
gcrMetricDataframe = pd.DataFrame()

for i in range(1, 2):
    
    # Generating random seed as integer up to 10000
    randomNums = np.random.randint(10000, size=(1))
    randomNum = randomNums[0]
    print(randomNum)

    wObject = WaterMark(randomNum)

    # All TIF files in the src_path are now imgs
    imgsSource = srcPth.glob('*.tif')
    
    # Counting the number of images in src_folder
    list = os.listdir(srcFolder)
    # -1 because there os one hidden file in directory
    numImages = len(list) - 1
    
    # Creating zeros Numpy Array and setting counter for results
    results = np.zeros([numImages, 9])
    counter = 0
        

    for img in imgsSource:
    
        # Read original image
        imgOriginal = WaterMark.imread(img)
        
        # Find impact factor within a PSNR range
        ImpactFactor = wObject.findImpactFactor(imgOriginal, rangePSNR = (35,40))
        
        # Mark image
        imgMarked = wObject.embedMark(imgOriginal, factor = ImpactFactor)
        
        # GCR Masking
        imgMasked = WaterMark.gcrMasking(imgOriginal, imgMarked)
        
        
        #CROPPING
        #CROPPING
        #CROPPING
        
        # Width
        lx = imgMasked.shape[0]
        # Height
        ly = imgMasked.shape[1]
        
        # Generating cropped image
        # 1/8 from each side (result is by 1/4 smaller than original)
        imgCropped = imgMarked[lx // 8: - lx // 8, ly // 8: - ly // 8]
        
        
        #SCALING
        #SCALING
        #SCALING
        
        # Wanted value of scaled image (in pixels)
        height = 768
        width = 768
        
        # Generating scaled image
        imgScaled = cv2.resize(imgMasked, (width, height))

        
        #ROTATION
        #ROTATION
        #ROTATION
        
        # Rotation angle
        angle = 90
        
        # Defines center of image
        rows, cols = imgMasked.shape[:2]
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        
        # Generating rotated image
        imgRotated = cv2.warpAffine(imgMasked, M, (cols, rows))    

        
        # AFFINE TRANSFORMATION
        # AFFINE TRANSFORMATION
        # AFFINE TRANSFORMATION
        
        # Defining points in original image
        point1a = [50, 50]
        point1b = [200, 50]
        point1c = [50, 200]
        # Defining points in transformed image
        point2a = [10, 100]
        point2b = [200, 50]
        point2c = [100, 250]
        
        rows, cols, ch = imgMasked.shape
        # Compiling original points into single array
        pts1 = np.float32([point1a, point1b, point1c])
        # Compiling transformed points into single array
        pts2 = np.float32([point2a, point2b, point2c])
        M = cv2.getAffineTransform(pts1, pts2)
        
        #Generating affined image
        imgAffined = cv2.warpAffine(imgMasked, M, (cols, rows))
       
        
        # 2D CONVOLUTION
        # 2D CONVOLUTION
        # 2D CONVOLUTION
        
        # Horizontal deviation (intensity)
        kernelX = 3
        # Vertical deviation (intensity)
        kernelY = 3
        
        kernelMultiplication = kernelX * kernelY
        kernel = np.ones((kernelX, kernelY), np.float32)/kernelMultiplication
        
        #Generating convoluted image
        imgConvoluted = cv2.filter2D(imgMasked, -1, kernel)
       
        
        # GAUSSIAN BLUR
        # GAUSSIAN BLUR
        # GAUSSIAN BLUR
        
        # Horizontal deviation (blur intensity)
        kernelX = 3
        # Vertical deviation (blur intensity)
        kernelY = 3
        
        # Generating blurred image
        imgBlurred = cv2.GaussianBlur(imgMasked, (kernelX, kernelY), cv2.BORDER_DEFAULT)
  
        
        # GAUSSIAN NOISE
        # GAUSSIAN NOISE
        # GAUSSIAN NOISE
        
        # Wanted value of rotation angle (in degrees)
        noiseAmount = 0.5
        
        # Generate Gaussian noise
        gauss = np.random.normal(0, noiseAmount, imgMasked.size)
        gauss = gauss.reshape(
            imgMasked.shape[0], imgMasked.shape[1], imgMasked.shape[2]).astype('uint8')
        
        # Generating Gaussian noised image
        imgNoised = cv2.add(imgMasked, gauss)
      
        
        metricOriginal = wObject.decodeMark(imgOriginal, 'CORR')
        results[counter, 0] = metricOriginal
        
        metricMarked = wObject.decodeMark(imgMarked, 'CORR')
        results[counter, 1] = metricMarked
        
        metricCropped = wObject.decodeMark(imgCropped, 'CORR')
        results[counter, 2] = metricCropped
        
        metricScaled = wObject.decodeMark(imgScaled, 'CORR')
        results[counter, 3] = metricScaled
        
        metricRotated = wObject.decodeMark(imgRotated, 'CORR')
        results[counter, 4] = metricRotated
        
        metricAffined = wObject.decodeMark(imgAffined, 'CORR')
        results[counter, 5] = metricAffined
        
        metricConvoluted = wObject.decodeMark(imgConvoluted, 'CORR')
        results[counter, 6] = metricConvoluted
        
        metricBlurred = wObject.decodeMark(imgBlurred, 'CORR')
        results[counter, 7] = metricBlurred
        
        metricNoised = wObject.decodeMark(imgNoised, 'CORR')
        results[counter, 8] = metricNoised
        
        counter += 1
        
        
        metricValues = ["Original", "Marked", "Cropped", "Scaled", "Rotated", "Affined", "2D Convoluted", "Blurred", "Noised"]
        metricSeeds = [f"{randomNum}", f"{randomNum}", f"{randomNum}", f"{randomNum}", f"{randomNum}", f"{randomNum}", f"{randomNum}", f"{randomNum}", f"{randomNum}"]
        
        finalMetric = pd.DataFrame(results, columns = [metricSeeds, metricValues])
    

    gcrMetricDataframe = pd.concat([metricDataframe, finalMetric], axis = 1)

In [ ]:
# Show Pandas DataFrame
metricDataframe

In [ ]:
gcrMetricDataframe

In [ ]:
stackedDataframe = metricDataframe.stack(0)

In [ ]:
stackedDataframe

In [ ]:
describeDataframe = metricDataframe.describe()

In [ ]:
describeDataframe

In [ ]:
meanDataframe = describeDataframe.iloc[1]

In [ ]:
meanDataframe

In [ ]:
plt.style.use('fivethirtyeight')

stackedDataframe.plot(kind = 'bar', stacked = False, legend = True)
plt.show()

In [ ]:
stackedDataframe.plot.hist(legend = True)